### Bonus Excercise - Efficiently Inefficient Financial Markets - Summer term 2025
 - Student name: Hanh Nguyen Nguyen
 - Student ID: 3167744

**Loading package**

In [3]:
import pandas as pd
import numpy as np

**Loading data**

All the files are in the current working directory

The `factor` data frame is retrieved from webpage of Kenneth R. French, section "Developed Markets Factors and Returns", file "Fama/French Developed 5 Factors [Daily]"

In [5]:
factor = pd.read_csv('Developed_5_Factors_Daily.csv', header=2)

#Format the Date col
factor.rename(columns={'Unnamed: 0':'Date'},inplace=True)
factor['Date'] = pd.to_datetime(factor['Date'], format='%Y%m%d')

#Filter the factor data in last 25 years only
factor = factor[factor['Date']>='2000-01-01']
factor.set_index('Date',inplace=True)

#Convert percentage values to decimals (e.g., 0.21% → 0.0021)
factor = factor/100

factor.head(10)

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2000-01-03,0.0021,0.0067,-0.0077,-0.0031,-0.0079,0.0002
2000-01-04,-0.0318,0.0104,0.0146,-0.0006,0.0063,0.0002
2000-01-05,-0.0151,0.0035,0.0137,-0.0031,0.0163,0.0002
2000-01-06,-0.0102,0.0039,0.0102,-0.0016,0.0101,0.0002
2000-01-07,0.0208,-0.0073,-0.0098,-0.0023,-0.0019,0.0002
2000-01-10,0.0125,0.0020,-0.0139,-0.0050,-0.0049,0.0002
2000-01-11,-0.0015,0.0013,-0.0018,0.0079,-0.0041,0.0002
2000-01-12,-0.0056,0.0019,0.0014,0.0028,0.0064,0.0002
2000-01-13,0.0084,0.0023,-0.0053,-0.0059,-0.0068,0.0002


The `price` data file is from Wuecampus, including  historical data on the closing log prices of a hedge fund spanning the past 25 years.

In [7]:
price = pd.read_csv('Nguyen_Hanh Nguyen.csv',index_col=0)
price.set_index('Date',inplace=True)
price.index = pd.to_datetime(price.index)
price.head(10)

,Log_Price_HF
Date,
2000-01-03,8.843644
2000-01-04,8.804754
2000-01-05,8.785065
2000-01-06,8.771407
2000-01-07,8.780288
2000-01-10,8.795992
2000-01-11,8.782467
2000-01-12,8.784597
2000-01-13,8.784386


The `ret` data frame includes daily return of the hedge fund  $r_t = logprice_t - logprice_{t-1}$

In [9]:
ret = pd.DataFrame()
ret['Return_HF'] = price['Log_Price_HF'].diff()
ret.index = price.index
ret.head(10)

,Return_HF
Date,
2000-01-03,NaN
2000-01-04,-0.038890
2000-01-05,-0.019689
2000-01-06,-0.013658
2000-01-07,0.008881
2000-01-10,0.015704
2000-01-11,-0.013525
2000-01-12,0.002130
2000-01-13,-0.000211


The `excess_return` data frame is including excess return, by subtracting `ret` to risk free rate `RF` from `factor` file

In [11]:
excess_return = pd.DataFrame()
excess_return = ret.sub(factor.loc[:,"RF"], axis=0)
excess_return.head(10)

,Return_HF
Date,
2000-01-03,NaN
2000-01-04,-0.039090
2000-01-05,-0.019889
2000-01-06,-0.013858
2000-01-07,0.008681
2000-01-10,0.015504
2000-01-11,-0.013725
2000-01-12,0.001930
2000-01-13,-0.000411


##### 1. Factor Loadings and Annualized Alpha Calculation:

In [13]:
import statsmodels.api as sm

 (a) The 1-factor market model
<center> $R_t^e = \alpha + \beta R_t^{M,e} + \epsilon_t$ </center>

In [15]:
y = excess_return.loc[:,'Return_HF']
x = factor.loc[:,'Mkt-RF']

#Remove missing values NaN in y (Excess return) and match index with x (Factor)
y = y.dropna()
x = x.loc[y.index]

one_factor_regression = sm.OLS(y, sm.add_constant(x)).fit()

one_factor_alpha = round(one_factor_regression.params.loc["const"] * 252 * 100, 2)

 (b) The Fama-French 3-factor model with SMB and HML
<center> $ R_t^e = \alpha + \beta^M R_t^{M,e} + \beta^{SMB} R_t^{SMB} +\beta^{HML} R_t^{HML} + \epsilon_t $ </center>

In [17]:
x = factor.loc[:,['Mkt-RF','SMB','HML']].loc[y.index]

three_factor_regression = sm.OLS(y, sm.add_constant(x)).fit()
three_factor_alpha = round(three_factor_regression.params.loc["const"] * 252 * 100,2)

 (c) The 5-factor model consisting of the FF 3-factor model augmented with RMW and CMA.

 <center> $ R_t^e = \alpha + \beta^M R_t^{M,e} + \beta^{SMB} R_t^{SMB} +\beta^{HML} R_t^{HML} + \beta^{RMW} R_t^{RMW} + \beta^{CMA} R_t^{CMA} + \epsilon_t $ </center>

In [19]:
x = factor.loc[:,['Mkt-RF','SMB','HML','RMW','CMA']].loc[y.index]

five_factor_regression = sm.OLS(y, sm.add_constant(x)).fit()
five_factor_alpha = round(five_factor_regression.params.loc["const"] * 252 * 100,2)

In [20]:
# special package to present regression tables
from stargazer.stargazer import Stargazer

In [21]:
#Represent regression result
table = Stargazer([one_factor_regression,three_factor_regression,five_factor_regression])
table.add_line("Annualized Alpha (in %)", [one_factor_alpha, three_factor_alpha,five_factor_alpha])
table.covariate_order(['const', 'Mkt-RF','SMB','HML','RMW','CMA'])
table

**2. Interpretation:**  
*Interpret the factor loadings obtained from the regressions. What insights do these loadings provide about the hedge fund’s investment style?*
- Market beta of the HF is positive and significant, indicating strong market exposure, i.e. managers tend to be more long than short in the market.
- The multi-factor model shows that, managers tend to prefer large stocks (SMB<0), value stocks with high book-to-market ratio (HML > 0) and preference for conservative investment strategies (CMA<0)
- RMW is insignificant, so profitability is not a key factor for the fund.
- Annualized alpha is negative (-5.51% to -6.66%), indicating no abnormal returns after risk adjustment.

**3. Comparison:**  
*Compare the alpha derived from the multivariate regressions to the alpha obtained from the univariate market regression. Discuss the differences in interpretation between the univariate and multivariate alphas, particularly in the context of the explanatory power*

- Univariate annualized alpha amounts to -5.51% at a significance 5%.  
- Multivariate annualized alpha amounts to -6.66% and -6.25% at a significance 1%.    
-> Multivariate alpha is much lower since part of the performance is explained the factor exposures.

- R-squared increases with more factors, indicating improved explanatory power but still moderate overall.

Overall, the fund is market-sensitive, favors large-value, conservative stocks, and does not outperform risk-adjusted benchmarks.